In [1]:
import numpy as np
import pickle
import keras.backend as K

from keras.layers import  GlobalAveragePooling1D, Input, Activation, MaxPooling1D, BatchNormalization, Dense, Dropout, Conv1D,GlobalMaxPooling1D
from keras.layers import GRU,AveragePooling1D,CuDNNGRU
from keras.layers.merge import Concatenate
from keras.models import Model 
from keras.callbacks import EarlyStopping,ModelCheckpoint


import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
import os


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)

Using TensorFlow backend.
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xhh/anaconda3/envs/py36/lib/p

In [2]:
import numpy as np
alphabet = np.array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                     'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])

def label_sequence(line, MAX_SEQ_LEN, smi_ch_ind):
	X = np.zeros(MAX_SEQ_LEN)

	for i, ch in enumerate(line[:MAX_SEQ_LEN]):
		X[i] = smi_ch_ind[ch]

	return X #.tolist()

def letter_one_hot(aa):
    one_hot = np.zeros(20)
    for idx, letter in enumerate(alphabet):
        if aa == letter:
            one_hot[idx] = 1
            return one_hot


# Convert an entire protein to one-hot representation.
def protein_one_hot(protein_sequence, MAX_SEQ_LEN):
    #  Remove non-specific AA codes (very few are actually present in this dataset)
    protein_sequence = protein_sequence.replace('B', '')
    protein_sequence = protein_sequence.replace('J', '')
    protein_sequence = protein_sequence.replace('O', '')
    protein_sequence = protein_sequence.replace('U', '')
    protein_sequence = protein_sequence.replace('X', '')
    protein_sequence = protein_sequence.replace('Z', '')
    one_hot_seq = np.zeros( (MAX_SEQ_LEN, 20))
    for idx, aa in enumerate(protein_sequence[:MAX_SEQ_LEN]):
        one_hot_seq[idx, :] = letter_one_hot(aa)
    return one_hot_seq


In [3]:
import keras
feature_len = 768
max_go_len = 256
max_seq_len = 1000

from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di



class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,  ppi_pair_file, batch_size=128):
        'Initialization' 
        self.batch_size = batch_size
        self.ppi_pair_file = ppi_pair_file
         
        self.max_seqlen = max_seq_len
        self.max_golen = max_go_len
         
        self.protein2seq = load_dict('ec2prot2seq.pkl')
        self.read_ppi()
        self.protein2onehot = {}
        self.onehot_seqs()
         
        self.on_epoch_end()
    
    def read_ppi(self):
        with open(self.ppi_pair_file, 'r') as f:
            self.ppi_pairs  =  f.readlines()
    
    def onehot_seqs(self):
        for key, value in self.protein2seq.items():
            self.protein2onehot[key] =  protein_one_hot(value, self.max_seqlen) 
            
    
   

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.ppi_pairs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.ppi_pairs))
         
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

         
        X_seq1 = np.empty((self.batch_size, self.max_seqlen,20))
        X_seq2 = np.empty((self.batch_size, self.max_seqlen,20))
        y = np.empty((self.batch_size))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split('\t')
            if label == '+':
                y[i] = 1
            else:
                y[i] = 0
            X_seq1[i] =  self.protein2onehot[p1]
            X_seq2[i] =  self.protein2onehot[p2]
             
            
            
        return [  X_seq1, X_seq2] ,  y



    def all_data(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

         
        X_seq1 = np.empty((len(list_IDs_temp), self.max_seqlen,20))

         
        X_seq2 = np.empty((len(list_IDs_temp), self.max_seqlen,20))
        y = np.empty((len(list_IDs_temp)))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split('\t')
            if label == '+':
                y[i] = 1
            else:
                y[i] = 0
            X_seq1[i] =  self.protein2onehot[p1]
            X_seq2[i] =  self.protein2onehot[p2]
           
        return [  X_seq1, X_seq2] ,  y

In [4]:
from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer


def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        # todo: check that this is correct
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True,
                 return_attention=False,
                 **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Note: The layer has been tested with Keras 1.x
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2 - Get the attention scores
            hidden = LSTM(64, return_sequences=True)(words)
            sentence, word_scores = Attention(return_attention=True)(hidden)
        """
        self.supports_masking = True
        self.return_attention = return_attention
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        eij = dot_product(x, self.W)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        weighted_input = x * K.expand_dims(a)

        result = K.sum(weighted_input, axis=1)

        if self.return_attention:
            return [result, a]
        return result

    def compute_output_shape(self, input_shape):
        if self.return_attention:
            return [(input_shape[0], input_shape[-1]),
                    (input_shape[0], input_shape[1])]
        else:
            return input_shape[0], input_shape[-1]

In [5]:
from keras.layers import   Embedding
from keras.layers import  GRU, Bidirectional, CuDNNGRU, Lambda, Flatten
from keras.layers.merge import concatenate
def inception_block(input_tensor, output_size):
    """"""
    con1d_filters = int(output_size/4)
    y = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x1 = Conv1D(con1d_filters, 5, activation="relu", padding='same')(y)

    y = Conv1D(con1d_filters, 1, activation="relu", padding='valid')(input_tensor)
    x2 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(y)

    x3 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x4 = Conv1D(con1d_filters, 1, activation="relu", padding='same')(input_tensor)

    y = Concatenate()([x1, x2, x3, x4])
#     y = MaxPooling1D(2)(mix0)
    # y = AveragePooling1D()(mix0)
#     y = BatchNormalization()(y)

    return y

def create_share_model():
    con_filters = 128
    X_input = Input(shape=(max_seq_len,20))
    # text-CNN
#     卷积池化-全连接 ---拼接-全连接  -双向GRU-全连接
    cnn = Conv1D(128, 3)(X_input)
#     cnn = BatchNormalization()(cnn)
    cnn = Activation('relu')(cnn)
     
    cnn = GlobalAveragePooling1D()(cnn)
    
    
    model = Model(X_input, cnn)
    return model


def build_siamese_model():
    left_input_seq = Input(shape=(max_seq_len,20))  
    right_input_seq = Input(shape=(max_seq_len,20))
    
    siamese_a = create_share_model()
    siamese_b = create_share_model()
    

    encoded_l = siamese_a(left_input_seq)
    encoded_r = siamese_b(right_input_seq)
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
#     dense = Dense(128,activation='relu')(L1_distance)
    prediction = Dense(1,activation='sigmoid')(L1_distance)

    siamese_net = Model(inputs=[left_input_seq,right_input_seq],outputs=prediction)
    
    siamese_net.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return siamese_net

def build_model():
    con_filters = 128
    
    left_input_seq = Input(shape=(max_seq_len,20))  
    right_input_seq = Input(shape=(max_seq_len,20))
    
    x = inception_block(left_input_seq,con_filters )
    x = Dropout(0.3)(x)
    x_gru = Bidirectional(CuDNNGRU(64, return_sequences=True))(left_input_seq)
    x_gru = Dropout(0.3)(x_gru)
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    left_x_seq = Concatenate()([x_a  , x_b, x_gru_a, x_gru_b])
    left_x_seq = Dense(256, activation='relu')(left_x_seq)
     
 
    x = inception_block(right_input_seq,con_filters )
    x = Dropout(0.3)(x)
    x_gru =Bidirectional(CuDNNGRU(64, return_sequences=True))(right_input_seq)
    x_gru = Dropout(0.3)(x_gru)
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)  
    right_x_seq = Concatenate()([x_a  , x_b, x_gru_a, x_gru_b])
    right_x_seq = Dense(256, activation='relu')(right_x_seq)
     
    x =   Concatenate()([left_x_seq, right_x_seq])
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.1)(x)
  
    x = Dense(1)(x)
    output = Activation('sigmoid')(x)
    # model = Model([left_input_go, right_input_go], output)
  
    model = Model([left_input_seq, right_input_seq], output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = build_model()
model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000, 20)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1000, 20)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1000, 32)     1952        input_1[0][0]                    
___________________________________________________________________________

In [6]:
from sklearn.model_selection import StratifiedKFold
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
dataset_name = 'EC2'
for rep in range(3):
    n_splits = 10
    TPRs =  np.zeros(n_splits)
    FPRs = np.zeros(n_splits)
    Precs = np.zeros(n_splits)
    ACCs = np.zeros(n_splits)
    F1s = np.zeros(n_splits)
    MCCs = np.zeros(n_splits)
    AUCs = np.zeros(n_splits)
    count = 0
    for split in range(n_splits):
        train_pairs_file = 'CV/train'+str(rep)+'-'+str(split)
        test_pairs_file = 'CV/test'+str(rep)+'-'+str(split)
        valid_pairs_file = 'CV/valid'+str(rep)+'-'+str(split)

        batch_size = 128
        train_generator = DataGenerator(   train_pairs_file,batch_size = batch_size )
        test_generator = DataGenerator(   test_pairs_file,batch_size = batch_size)
        valid_generator = DataGenerator(   valid_pairs_file,batch_size = batch_size)
         
        # model = build_model_without_att()
        model = build_model()
        save_model_name = 'CV/ec2_seq'+str(rep)+'-'+str(split) + '.hdf5'
        
        earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
        save_checkpoint = ModelCheckpoint(save_model_name, save_best_only=True, monitor='val_loss', mode='min', save_weights_only=True)

         
        # validation_data = (valid_X, valid_Y),  verbose=1,callbacks=[earlyStopping, save_checkpoint]
        hist = model.fit_generator(generator=train_generator,
                    validation_data=valid_generator,
                    epochs = 100,verbose=1,callbacks=[earlyStopping, save_checkpoint] )
         
        
        # model = load_model(save_model_name)
        model.load_weights(save_model_name)
        with open(test_pairs_file, 'r') as f:
            test_ppi_pairs  =  f.readlines()

        test_len = len(test_ppi_pairs) 
        list_IDs_temp = np.arange(test_len)

        test_x, y_test = test_generator.all_data(list_IDs_temp)

        y_pred_prob = model.predict(test_x)

       
        y_pred = (y_pred_prob > 0.5)
        auc = metrics.roc_auc_score(y_test, y_pred_prob) 
        f1 = f1_score(y_test, y_pred)
        pre = precision_score(y_test, y_pred)
        acc = accuracy_score(y_test, y_pred)

        precision, recall, _thresholds = metrics.precision_recall_curve(y_test, y_pred_prob)
        pr_auc = metrics.auc(recall, precision)
        mcc = matthews_corrcoef(y_test, y_pred)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total=tn+fp+fn+tp
        sen = float(tp)/float(tp+fn)
        sps = float(tn)/float((tn+fp))

        tpr = float(tp)/float(tp+fn)
        fpr = float(fp)/float((tn+fp))
        print('--------------------------\n')
        print ('AUC: %f' % auc)
        print ('ACC: %f' % acc) 
        # print("PRAUC: %f" % pr_auc)
        print ('MCC : %f' % mcc)
        # print ('SEN: %f' % sen)
        # print ('SEP: %f' % sps)
        print('TPR:%f'%tpr)
        print('FPR:%f'%fpr)
        print('Pre:%f'%pre)
        print('F1:%f'%f1)
        print('--------------------------\n')
        TPRs[count] = tpr
        FPRs[count] = fpr
        Precs[count] =pre
        ACCs[count] =acc
        F1s[count] =f1
        MCCs[count] =mcc
        AUCs[count] =auc
        count += 1
    print ('mean AUC: %f' % np.mean(AUCs))
    print ('mean ACC: %f' % np.mean(ACCs)) 
    print ('mean MCC : %f' % np.mean(MCCs))
    print('mean TPR:%f'% np.mean(TPRs))
    print('mean FPR:%f'% np.mean(FPRs))
    print('mean Pre:%f'% np.mean(Precs))
    print('mean F1:%f'% np.mean(F1s))
    np.savez('ec2_seq_'+str(rep), AUCs=AUCs, ACCs=ACCs, MCCs=MCCs, TPRs = TPRs, FPRs=FPRs, Precs=Precs, F1s=F1s)

Epoch 1/100
13/13 [==============================] - 10s 793ms/step - loss: 0.7216 - acc: 0.5276 - val_loss: 0.6740 - val_acc: 0.7839
Epoch 2/100
13/13 [==============================] - 4s 346ms/step - loss: 0.6482 - acc: 0.6418 - val_loss: 0.6222 - val_acc: 0.6016
Epoch 3/100
13/13 [==============================] - 4s 333ms/step - loss: 0.5601 - acc: 0.7248 - val_loss: 0.5093 - val_acc: 0.7812
Epoch 4/100
13/13 [==============================] - 4s 343ms/step - loss: 0.5031 - acc: 0.7650 - val_loss: 0.4955 - val_acc: 0.8125
Epoch 5/100
13/13 [==============================] - 4s 333ms/step - loss: 0.4534 - acc: 0.7939 - val_loss: 0.4363 - val_acc: 0.8203
Epoch 6/100
13/13 [==============================] - 4s 329ms/step - loss: 0.4247 - acc: 0.8005 - val_loss: 0.4457 - val_acc: 0.8177
Epoch 7/100
13/13 [==============================] - 4s 333ms/step - loss: 0.4019 - acc: 0.8197 - val_loss: 0.4242 - val_acc: 0.8229
Epoch 8/100
13/13 [==============================] - 4s 331ms/step -

13/13 [==============================] - 5s 349ms/step - loss: 0.4571 - acc: 0.7879 - val_loss: 0.4421 - val_acc: 0.8073
Epoch 8/100
13/13 [==============================] - 4s 334ms/step - loss: 0.4133 - acc: 0.8113 - val_loss: 0.4383 - val_acc: 0.8229
Epoch 9/100
13/13 [==============================] - 5s 360ms/step - loss: 0.3908 - acc: 0.8245 - val_loss: 0.4086 - val_acc: 0.8333
Epoch 10/100
13/13 [==============================] - 5s 346ms/step - loss: 0.3912 - acc: 0.8161 - val_loss: 0.4244 - val_acc: 0.8307
Epoch 11/100
13/13 [==============================] - 4s 338ms/step - loss: 0.3800 - acc: 0.8281 - val_loss: 0.4169 - val_acc: 0.8385
Epoch 12/100
13/13 [==============================] - 4s 344ms/step - loss: 0.3614 - acc: 0.8347 - val_loss: 0.3958 - val_acc: 0.8359
Epoch 13/100
13/13 [==============================] - 4s 336ms/step - loss: 0.3566 - acc: 0.8365 - val_loss: 0.3966 - val_acc: 0.8438
Epoch 14/100
13/13 [==============================] - 4s 314ms/step - loss: 0

13/13 [==============================] - 10s 735ms/step - loss: 0.7079 - acc: 0.5252 - val_loss: 0.6759 - val_acc: 0.7734
Epoch 2/100
13/13 [==============================] - 5s 355ms/step - loss: 0.6476 - acc: 0.6502 - val_loss: 0.5955 - val_acc: 0.7552
Epoch 3/100
13/13 [==============================] - 5s 366ms/step - loss: 0.5492 - acc: 0.7368 - val_loss: 0.5315 - val_acc: 0.7969
Epoch 4/100
13/13 [==============================] - 5s 363ms/step - loss: 0.4953 - acc: 0.7638 - val_loss: 0.4768 - val_acc: 0.8177
Epoch 5/100
13/13 [==============================] - 5s 364ms/step - loss: 0.4631 - acc: 0.7891 - val_loss: 0.4525 - val_acc: 0.8411
Epoch 6/100
13/13 [==============================] - 4s 332ms/step - loss: 0.4301 - acc: 0.8029 - val_loss: 0.4312 - val_acc: 0.8438
Epoch 7/100
13/13 [==============================] - 4s 328ms/step - loss: 0.4045 - acc: 0.8161 - val_loss: 0.4504 - val_acc: 0.8255
Epoch 8/100
13/13 [==============================] - 4s 332ms/step - loss: 0.415

13/13 [==============================] - 5s 350ms/step - loss: 0.4539 - acc: 0.8299 - val_loss: 0.4222 - val_acc: 0.8281
Epoch 30/100
13/13 [==============================] - 5s 357ms/step - loss: 0.3437 - acc: 0.8534 - val_loss: 0.4145 - val_acc: 0.7943
Epoch 31/100
13/13 [==============================] - 5s 347ms/step - loss: 0.2331 - acc: 0.9050 - val_loss: 0.4470 - val_acc: 0.7839
Epoch 32/100
13/13 [==============================] - 5s 358ms/step - loss: 0.1871 - acc: 0.9207 - val_loss: 0.4578 - val_acc: 0.7943
Epoch 33/100
13/13 [==============================] - 5s 347ms/step - loss: 0.1583 - acc: 0.9351 - val_loss: 0.5375 - val_acc: 0.7474
Epoch 34/100
13/13 [==============================] - 4s 341ms/step - loss: 0.2129 - acc: 0.9069 - val_loss: 0.3869 - val_acc: 0.8646
Epoch 35/100
13/13 [==============================] - 5s 355ms/step - loss: 0.1733 - acc: 0.9315 - val_loss: 0.3915 - val_acc: 0.8307
--------------------------

AUC: 0.976622
ACC: 0.897436
MCC : 0.809582
TPR:

13/13 [==============================] - 5s 356ms/step - loss: 0.2538 - acc: 0.8828 - val_loss: 0.4219 - val_acc: 0.8099
Epoch 21/100
13/13 [==============================] - 5s 354ms/step - loss: 0.2193 - acc: 0.9099 - val_loss: 0.5265 - val_acc: 0.7396
Epoch 22/100
13/13 [==============================] - 4s 322ms/step - loss: 0.2290 - acc: 0.9026 - val_loss: 0.4285 - val_acc: 0.8125
Epoch 23/100
13/13 [==============================] - 4s 322ms/step - loss: 0.2274 - acc: 0.9032 - val_loss: 0.4102 - val_acc: 0.8151
Epoch 24/100
13/13 [==============================] - 5s 346ms/step - loss: 0.2732 - acc: 0.8840 - val_loss: 0.6338 - val_acc: 0.6536
--------------------------

AUC: 0.900276
ACC: 0.814655
MCC : 0.669819
TPR:0.605505
FPR:0.000000
Pre:1.000000
F1:0.754286
--------------------------

Epoch 1/100
13/13 [==============================] - 12s 887ms/step - loss: 0.7024 - acc: 0.5270 - val_loss: 0.6709 - val_acc: 0.6797
Epoch 2/100
13/13 [==============================] - 4s 318

...

13/13 [==============================] - 3s 238ms/step - loss: 0.3244 - acc: 0.8540 - val_loss: 0.4445 - val_acc: 0.7292
Epoch 14/100
13/13 [==============================] - 3s 240ms/step - loss: 0.3123 - acc: 0.8690 - val_loss: 0.4061 - val_acc: 0.8333
Epoch 15/100
13/13 [==============================] - 3s 241ms/step - loss: 0.3071 - acc: 0.8666 - val_loss: 0.3982 - val_acc: 0.8177
Epoch 16/100
13/13 [==============================] - 3s 235ms/step - loss: 0.2929 - acc: 0.8708 - val_loss: 0.4290 - val_acc: 0.7917
Epoch 17/100
13/13 [==============================] - 3s 243ms/step - loss: 0.2591 - acc: 0.8876 - val_loss: 0.4125 - val_acc: 0.8073
Epoch 18/100
13/13 [==============================] - 3s 237ms/step - loss: 0.2407 - acc: 0.8996 - val_loss: 0.4344 - val_acc: 0.8073
Epoch 19/100
13/13 [==============================] - 3s 236ms/step - loss: 0.2956 - acc: 0.8708 - val_loss: 0.4356 - val_acc: 0.7839
Epoch 20/100
13/13 [==============================] - 3s 240ms/step - loss:

13/13 [==============================] - 3s 203ms/step - loss: 0.3464 - acc: 0.8492 - val_loss: 0.3545 - val_acc: 0.8411
Epoch 24/100
13/13 [==============================] - 3s 223ms/step - loss: 0.3176 - acc: 0.8630 - val_loss: 0.3358 - val_acc: 0.8542
Epoch 25/100
13/13 [==============================] - 3s 231ms/step - loss: 0.2764 - acc: 0.8828 - val_loss: 0.3455 - val_acc: 0.8411
Epoch 26/100
13/13 [==============================] - 3s 234ms/step - loss: 0.3172 - acc: 0.8618 - val_loss: 0.3465 - val_acc: 0.8438
Epoch 27/100
13/13 [==============================] - 3s 240ms/step - loss: 0.2836 - acc: 0.8726 - val_loss: 0.3574 - val_acc: 0.8542
Epoch 28/100
13/13 [==============================] - 3s 233ms/step - loss: 0.2465 - acc: 0.8990 - val_loss: 0.3611 - val_acc: 0.8177
Epoch 29/100
13/13 [==============================] - 3s 238ms/step - loss: 0.2450 - acc: 0.8912 - val_loss: 0.3368 - val_acc: 0.8620
--------------------------

AUC: 0.926343
ACC: 0.846154
MCC : 0.724718
TPR:

13/13 [==============================] - 3s 244ms/step - loss: 0.2163 - acc: 0.9062 - val_loss: 0.3623 - val_acc: 0.8229
Epoch 29/100
13/13 [==============================] - 3s 245ms/step - loss: 0.2243 - acc: 0.8948 - val_loss: 0.3650 - val_acc: 0.8281
Epoch 30/100
13/13 [==============================] - 3s 237ms/step - loss: 0.1680 - acc: 0.9393 - val_loss: 0.3896 - val_acc: 0.8333
Epoch 31/100
13/13 [==============================] - 3s 242ms/step - loss: 0.1909 - acc: 0.9141 - val_loss: 0.4748 - val_acc: 0.8307
Epoch 32/100
13/13 [==============================] - 3s 245ms/step - loss: 0.1860 - acc: 0.9225 - val_loss: 0.4061 - val_acc: 0.7969
Epoch 33/100
13/13 [==============================] - 3s 242ms/step - loss: 0.1558 - acc: 0.9339 - val_loss: 0.4145 - val_acc: 0.8099
Epoch 34/100
13/13 [==============================] - 3s 243ms/step - loss: 0.1250 - acc: 0.9519 - val_loss: 0.4197 - val_acc: 0.8177
Epoch 35/100
13/13 [==============================] - 3s 241ms/step - loss:

13/13 [==============================] - 3s 244ms/step - loss: 0.3377 - acc: 0.8498 - val_loss: 0.4278 - val_acc: 0.8151
Epoch 23/100
13/13 [==============================] - 3s 243ms/step - loss: 0.3243 - acc: 0.8636 - val_loss: 0.4148 - val_acc: 0.8125
Epoch 24/100
13/13 [==============================] - 3s 247ms/step - loss: 0.2588 - acc: 0.8840 - val_loss: 0.4267 - val_acc: 0.7812
Epoch 25/100
13/13 [==============================] - 3s 248ms/step - loss: 0.2444 - acc: 0.8930 - val_loss: 0.4083 - val_acc: 0.7891
Epoch 26/100
13/13 [==============================] - 3s 247ms/step - loss: 0.2060 - acc: 0.9111 - val_loss: 0.5138 - val_acc: 0.7552
Epoch 27/100
13/13 [==============================] - 3s 250ms/step - loss: 0.2551 - acc: 0.8912 - val_loss: 0.4760 - val_acc: 0.8177
Epoch 28/100
13/13 [==============================] - 3s 250ms/step - loss: 0.2763 - acc: 0.8840 - val_loss: 0.4257 - val_acc: 0.8047
Epoch 29/100
13/13 [==============================] - 3s 245ms/step - loss:

13/13 [==============================] - 4s 344ms/step - loss: 0.1568 - acc: 0.9411 - val_loss: 0.6358 - val_acc: 0.6901
Epoch 27/100
13/13 [==============================] - 4s 346ms/step - loss: 0.2620 - acc: 0.8846 - val_loss: 0.4503 - val_acc: 0.7604
--------------------------

AUC: 0.933183
ACC: 0.862069
MCC : 0.735447
TPR:0.723810
FPR:0.023622
Pre:0.962025
F1:0.826087
--------------------------

Epoch 1/100
13/13 [==============================] - 20s 2s/step - loss: 0.7325 - acc: 0.5138 - val_loss: 0.6922 - val_acc: 0.4740
Epoch 2/100
13/13 [==============================] - 5s 376ms/step - loss: 0.6672 - acc: 0.5799 - val_loss: 0.6555 - val_acc: 0.6354
Epoch 3/100
13/13 [==============================] - 5s 362ms/step - loss: 0.5837 - acc: 0.7284 - val_loss: 0.5446 - val_acc: 0.7526
Epoch 4/100
13/13 [==============================] - 5s 357ms/step - loss: 0.5099 - acc: 0.7614 - val_loss: 0.5839 - val_acc: 0.7396
Epoch 5/100
13/13 [==============================] - 5s 369ms/ste

In [7]:
import numpy as np 
results1 =   np.load( 'ec2_seq_0.npz')
results2 =   np.load( 'ec2_seq_1.npz')
results3 =   np.load( 'ec2_seq_2.npz')
print ('mean AUC: %f' %  ( (np.mean( results1[ 'AUCs' ] )  + np.mean(  results2[ 'AUCs' ] )  + np.mean(results3[ 'AUCs' ]))/3     ) )
print ('mean ACC: %f' %   ( (np.mean( results1[ 'ACCs' ] )  + np.mean(  results2[ 'ACCs' ] )  + np.mean(results3[ 'ACCs' ]))/3) )
print ('mean MCC : %f' %  (  (np.mean( results1[ 'MCCs' ] )  + np.mean(  results2[ 'MCCs' ] )  + np.mean(results3[ 'MCCs' ])     )/3))
print('mean TPR:%f'%    ((np.mean( results1[ 'TPRs' ] )  + np.mean(  results2[ 'TPRs' ] )  + np.mean(results3[ 'TPRs' ])     )/3))
print('mean FPR:%f'%   ( (np.mean( results1[ 'FPRs' ] )  + np.mean(  results2[ 'FPRs' ] )  + np.mean(results3[ 'FPRs' ])     )/3))
print('mean Pre:%f'%    ((np.mean( results1[ 'Precs' ] )  + np.mean(  results2[ 'Precs' ] )  + np.mean(results3[ 'Precs' ])     )/3))
print('mean F1:%f'%    ((np.mean( results1[ 'F1s' ] )  + np.mean(  results2[ 'F1s' ] )  + np.mean(results3[ 'F1s' ])     )/3))

mean AUC: 0.936394
mean ACC: 0.866314
mean MCC : 0.753634
mean TPR:0.753846
mean FPR:0.021557
mean Pre:0.974351
mean F1:0.847611
